In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler 
import seaborn as sns 
import os
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer

In [254]:
os.chdir(r'D:/CDAC MARCH 2024 AI/PML/assignments/4/')
dtest = pd.read_csv(r'test.csv')
dtrain = pd.read_csv(r'train.csv')

In [245]:

dtrain.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [227]:
idW = dtrain.groupby('Item_Identifier').Item_Weight.mean()
dtrain.shape
def fillItemWeight(row):
    #print(row['Item_Weight'])
    if row['Item_Weight']==np.nan or row['Item_Weight']==None:
        #print(row)
        row['Item_Weight'] = idW[row['Item_Identifier']]
    return row    
d_new = dtrain.apply(lambda r: fillItemWeight(r), axis=1)
#display(d_new)
#d_new.isna().sum(axis=0)

### Train Data Set

In [239]:
# imputing means in missing values in item_weight repective to their identifier
means = dtrain.groupby('Item_Identifier').Item_Weight.transform('mean')
display(means)
dtrain['Item_Weight'] = dtrain['Item_Weight'].fillna(means)
dtrain.isna().sum(axis=0)

0        9.300
1        5.920
2       17.500
3       19.200
4        8.930
         ...  
8518     6.865
8519     8.380
8520    10.600
8521     7.210
8522    14.800
Name: Item_Weight, Length: 8523, dtype: float64

Item_Identifier                 0
Item_Weight                     4
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

### Test Dataset

In [240]:
# imputing means in missing values in item_weight repective to their identifier
means = dtest.groupby('Item_Identifier').Item_Weight.transform('mean')
display(means)
dtest['Item_Weight'] = dtest['Item_Weight'].fillna(means)
dtest.isna().sum(axis=0)

0       20.750
1        8.300
2       14.600
3        7.315
4       13.600
         ...  
5676    10.500
5677     7.600
5678    10.000
5679    15.300
5680     9.500
Name: Item_Weight, Length: 5681, dtype: float64

Item_Identifier                 0
Item_Weight                    20
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [149]:
dtrain.shape

(8523, 11)

In [151]:
dtest.shape

(5681, 11)

In [255]:
#items_train = dtrain[['Item_Identifier','Item_Weight']].sort_values(by='Item_Identifier')
# items_train = dtrain[['Item_Identifier','Item_Weight']]
# items_test = dtest[['Item_Identifier','Item_Weight']]
# con = pd.concat([items_train, items_test], ignore_index=True, join='inner')


con = pd.concat([dtrain[['Item_Identifier','Item_Weight']],dtest[['Item_Identifier','Item_Weight']]], ignore_index=True, join='inner')

con.drop_duplicates(inplace=True)
con.dropna(inplace=True)
display(con.shape)

(1559, 2)

In [256]:
con.sort_values(by='Item_Identifier')
display(con.isna().sum(axis=0))
con.head()

Item_Identifier    0
Item_Weight        0
dtype: int64

,Item_Identifier,Item_Weight
0,FDA15,9.30
1,DRC01,5.92
2,FDN15,17.50
3,FDX07,19.20
4,NCD19,8.93


In [257]:
dtrain.drop('Item_Weight', axis=1, inplace=True)
#dtrain.dropna(inplace=True)
display(dtrain.shape)
dtest.drop('Item_Weight', axis=1, inplace=True)
display(dtest.shape)

(8523, 11)

(5681, 10)

In [258]:
### removing remaining 4
dtrain = pd.merge(dtrain,con, on='Item_Identifier')
dtest = pd.merge(dtest,con, on='Item_Identifier')
display(dtrain.shape)
display(dtest.shape)

(8523, 12)

(5681, 11)

In [259]:
display(dtrain.isna().sum(axis=0))
display(dtrain.head())
display(dtest.isna().sum(axis=0))
display(dtest.head())

Item_Identifier                 0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
Item_Weight                     0
dtype: int64

,Item_Identifier,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Weight
0,FDA15,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,9.3
1,FDA15,Low Fat,0.016055,Dairy,250.2092,OUT045,2002,NaN,Tier 2,Supermarket Type1,5976.2208,9.3
2,FDA15,Low Fat,0.016019,Dairy,248.5092,OUT035,2004,Small,Tier 2,Supermarket Type1,6474.2392,9.3
3,FDA15,Low Fat,0.016088,Dairy,249.6092,OUT018,2009,Medium,Tier 3,Supermarket Type2,5976.2208,9.3
4,FDA15,Low Fat,0.026818,Dairy,248.9092,OUT010,1998,NaN,Tier 3,Grocery Store,498.0184,9.3


Item_Identifier                 0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
Item_Weight                     0
dtype: int64

,Item_Identifier,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Weight
0,FDW58,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1,20.75
1,FDW58,Low Fat,0.007596,Snack Foods,104.4622,OUT017,2007,NaN,Tier 2,Supermarket Type1,20.75
2,FDW58,Low Fat,0.007584,Snack Foods,107.0622,OUT018,2009,Medium,Tier 3,Supermarket Type2,20.75
3,FDW58,Low Fat,0.000000,Snack Foods,105.9622,OUT046,1997,Small,Tier 1,Supermarket Type1,20.75
4,FDW58,Low Fat,0.007568,Snack Foods,105.8622,OUT045,2002,NaN,Tier 2,Supermarket Type1,20.75


### For outlet_size missing values

In [260]:
outlet_train = dtrain[['Outlet_Identifier', 'Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type']].sort_values(by='Outlet_Identifier')
outlet_test = dtest[['Outlet_Identifier', 'Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type']].sort_values(by='Outlet_Identifier')
con  = pd.concat([outlet_train, outlet_test], join='inner', ignore_index=True)
con.drop_duplicates(inplace=True)
#con.dropna(inplace=True)
display(con.shape)
display(con.head())

(10, 5)

,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,OUT010,1998,NaN,Tier 3,Grocery Store
555,OUT013,1987,High,Tier 3,Supermarket Type1
1487,OUT017,2007,NaN,Tier 2,Supermarket Type1
2413,OUT018,2009,Medium,Tier 3,Supermarket Type2
3341,OUT019,1985,Small,Tier 1,Grocery Store


In [261]:

dtrain.drop(['Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type'], axis=1, inplace=True)
dtrain = pd.merge(dtrain,con, on='Outlet_Identifier')
display(dtrain.shape)

dtest.drop(['Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type'], axis=1, inplace=True)
dtest = pd.merge(dtest,con, on='Outlet_Identifier')
display(dtest.shape)


(8523, 12)

(5681, 11)

In [268]:
dtrain.isna().sum(axis=0)
dtrain['Outlet_Size'].fillna(value='Small', inplace=True)
dtest['Outlet_Size'].fillna(value='Small', inplace=True)

In [252]:
display(dtrain.head())

,Item_Identifier,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Item_Outlet_Sales,Item_Weight,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDA15,Low Fat,0.016047,Dairy,249.8092,OUT049,3735.1380,9.30,1999,Medium,Tier 1,Supermarket Type1
1,FDN15,Low Fat,0.016760,Meat,141.6180,OUT049,2097.2700,17.50,1999,Medium,Tier 1,Supermarket Type1
2,FDX07,Regular,0.022954,Fruits and Vegetables,184.4950,OUT049,4028.0900,19.20,1999,Medium,Tier 1,Supermarket Type1
3,NCD19,LF,0.000000,Household,53.2614,OUT049,552.6140,8.93,1999,Medium,Tier 1,Supermarket Type1
4,FDP10,Low Fat,0.128289,Snack Foods,104.9622,OUT049,1164.4842,19.00,1999,Medium,Tier 1,Supermarket Type1


In [269]:
X_train = dtrain.drop(['Item_Identifier','Outlet_Identifier'], axis=1)
display(X_train.head())
X_test = dtest.drop(['Item_Identifier','Outlet_Identifier'],  axis=1)
display(X_test.head())

,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Item_Outlet_Sales,Item_Weight,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,Low Fat,0.016047,Dairy,249.8092,3735.1380,9.30,1999,Medium,Tier 1,Supermarket Type1
1,Low Fat,0.016760,Meat,141.6180,2097.2700,17.50,1999,Medium,Tier 1,Supermarket Type1
2,Regular,0.022954,Fruits and Vegetables,184.4950,4028.0900,19.20,1999,Medium,Tier 1,Supermarket Type1
3,LF,0.000000,Household,53.2614,552.6140,8.93,1999,Medium,Tier 1,Supermarket Type1
4,Low Fat,0.128289,Snack Foods,104.9622,1164.4842,19.00,1999,Medium,Tier 1,Supermarket Type1


,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Item_Weight,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,Low Fat,0.007565,Snack Foods,107.8622,20.750,1999,Medium,Tier 1,Supermarket Type1
1,Regular,0.038271,Dairy,85.9198,8.300,1999,Medium,Tier 1,Supermarket Type1
2,Low Fat,0.015884,Baking Goods,83.4592,9.195,1999,Medium,Tier 1,Supermarket Type1
3,Low Fat,0.005675,Baking Goods,184.1924,5.985,1999,Medium,Tier 1,Supermarket Type1
4,Low Fat,0.092738,Breads,122.3098,4.785,1999,Medium,Tier 1,Supermarket Type1


In [270]:
y_train = X_train['Item_Outlet_Sales']
X_train = X_train.drop('Item_Outlet_Sales',axis=1)
X_test = dtest

In [271]:
ohc = OneHotEncoder(sparse_output=False)
mk = make_column_transformer((ohc, make_column_selector(dtype_include=object)), ("passthrough",make_column_selector(dtype_include=['int64','float64'])),verbose_feature_names_out=False).set_output(transform='pandas')
X_train = mk.fit_transform(X_train)
X_test = mk.transform(X_test)

In [272]:
############################ Regression with catboost ###################################

from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV, KFold

cbgm = CatBoostRegressor(random_state=24, logging_level='Silent')
params = {"max_depth":[None, 2, 3, 4],
              "n_estimators": [25, 50, 100],
              "learning_rate": np.linspace(0.01, 0.9, 10)}

kfold = KFold(n_splits=5, shuffle=True, random_state=24)
#param = {"alpha": np.linspace(0.001,5,10),"l1_ratio":np.linspace(0.1,0.9,9)}

gcv_cat = GridSearchCV(cbgm,cv = kfold, param_grid=params,scoring="neg_root_mean_squared_error", n_jobs=-1)
gcv_cat.fit(X_train,y_train)
print(gcv_cat.best_score_)
print(gcv_cat.best_params_)
#cbgm.fit(X_train,y_train)
y_pred = gcv_cat.predict(X_test)

-1076.7067056682704
{'learning_rate': 0.10888888888888888, 'max_depth': 4, 'n_estimators': 50}


In [273]:
y_pred[y_pred < 0] =0
df = pd.DataFrame({'Item_Identifier':dtest['Item_Identifier'], 'Outlet_Identifier': dtest['Outlet_Identifier'], 'Item_Outlet_Sales':y_pred})
df.to_csv(r'D:/CDAC MARCH 2024 AI/PML/assignments/4/submission_4.csv', index=False)

In [ ]:
dtrain.isna().sum(axis=0)
dtrain.head()

In [84]:
dtrain.groupby('Outlet_Identifier').Outlet_Size.value_counts().index.values
max = dict(max)


array([('OUT013', 'High'), ('OUT018', 'Medium'), ('OUT019', 'Small'),
       ('OUT027', 'Medium'), ('OUT035', 'Small'), ('OUT046', 'Small'),
       ('OUT049', 'Medium')], dtype=object)

{'OUT013': 'High',
 'OUT018': 'Medium',
 'OUT019': 'Small',
 'OUT027': 'Medium',
 'OUT035': 'Small',
 'OUT046': 'Small',
 'OUT049': 'Medium'}

In [91]:
dtrain.groupby('Outlet_Identifier').Outlet_Size.max()

Outlet_Identifier
OUT010       NaN
OUT013      High
OUT017       NaN
OUT018    Medium
OUT019     Small
OUT027    Medium
OUT035     Small
OUT045       NaN
OUT046     Small
OUT049    Medium
Name: Outlet_Size, dtype: object

In [93]:
dtrain[dtrain['Outlet_Identifier']=='OUT010'].Outlet_Size

3       NaN
28      NaN
30      NaN
45      NaN
65      NaN
       ... 
8400    NaN
8432    NaN
8473    NaN
8486    NaN
8509    NaN
Name: Outlet_Size, Length: 555, dtype: object

In [86]:
# imputing means in missing values in outlet_size repective to their identifier
# max = dtrain.groupby('Outlet_Identifier').Outlet_Size.transform('max')
# display(max)
# dtrain['Outlet_Size'] = dtrain['Outlet_Size'].fillna(dtrain['Outlet_Identifier'].map(max))
# dtrain.isna().sum(axis=0)

Item_Identifier                 0
Item_Weight                     4
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [52]:
# imputing means in missing values in item_weight repective to their identifier
# means = dtrain.groupby('Item_Identifier').Item_Weight.transform('mean')
# display(means)
# dtrain['Item_Weight'] = dtrain['Item_Weight'].fillna(means)
# dtrain.isna().sum(axis=0)

0        9.300
1        5.920
2       17.500
3       19.200
4        8.930
         ...  
8518     6.865
8519     8.380
8520    10.600
8521     7.210
8522    14.800
Name: Item_Weight, Length: 8523, dtype: float64

Item_Identifier                 0
Item_Weight                     4
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [ ]:
# imputing means in missing values in outlet_size repective to their identifier
# means = dtrain.groupby('Outlet_Identifier').Item_Weight.transform('mean')
# display(means)
# dtrain['Outlet_Size'] = dtrain['Outlet_Size'].fillna(means)
# dtrain.isna().sum(axis=0)

In [26]:
display(dtrain.shape)
dt = dtrain.drop_duplicates()
display(dt.shape)

(8523, 12)

(8523, 12)

In [71]:
dtrain.drop(['Item_Identifier','Outlet_Identifier'], inplace=True, axis=1)
dtrain.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,9.30,Low Fat,0.016047,Dairy,249.8092,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,5.92,Regular,0.019278,Soft Drinks,48.2692,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,17.50,Low Fat,0.016760,Meat,141.6180,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,1998,NaN,Tier 3,Grocery Store,732.3800
4,8.93,Low Fat,0.000000,Household,53.8614,1987,High,Tier 3,Supermarket Type1,994.7052


In [72]:

dtest.drop(['Item_Identifier','Outlet_Identifier'], inplace=True, axis=1)
dtest.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,20.750,Low Fat,0.007565,Snack Foods,107.8622,1999,Medium,Tier 1,Supermarket Type1
1,8.300,reg,0.038428,Dairy,87.3198,2007,NaN,Tier 2,Supermarket Type1
2,14.600,Low Fat,0.099575,Others,241.7538,1998,NaN,Tier 3,Grocery Store
3,7.315,Low Fat,0.015388,Snack Foods,155.0340,2007,NaN,Tier 2,Supermarket Type1
4,NaN,Regular,0.118599,Dairy,234.2300,1985,Medium,Tier 3,Supermarket Type3


In [73]:
y_train = dtrain['Item_Outlet_Sales']
X_train = dtrain.drop('Item_Outlet_Sales',axis=1)



In [74]:
imp_num = SimpleImputer(strategy='mean').set_output(transform='pandas')
imp_cat = SimpleImputer(strategy='most_frequent').set_output(transform='pandas')
mc = make_column_transformer((imp_num, make_column_selector(dtype_include=[np.number])),(imp_cat,make_column_selector(dtype_include=object)),
                             verbose_feature_names_out=False).set_output(transform='pandas')
X_train = mc.fit_transform(X_train)
X_test = mc.transform(dtest)

In [75]:
# encoding 
ohc = OneHotEncoder(sparse_output=False)
mk = make_column_transformer((ohc, make_column_selector(dtype_include=object)), ("passthrough",make_column_selector(dtype_include=['int64','float64'])),verbose_feature_names_out=False).set_output(transform='pandas')
X_train = mk.fit_transform(X_train)
X_test = mk.transform(X_test)

In [76]:
X_train

,Item_Fat_Content_LF,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Fat_Content_reg,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,...,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,9.300,0.016047,249.8092,1999.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,5.920,0.019278,48.2692,2009.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,17.500,0.016760,141.6180,1999.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,19.200,0.000000,182.0950,1998.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,8.930,0.000000,53.8614,1987.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,6.865,0.056783,214.5218,1987.0
8519,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,8.380,0.046982,108.1570,2002.0
8520,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,10.600,0.035186,85.1224,2004.0
8521,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,7.210,0.145221,103.1332,2009.0


In [77]:
X_test

,Item_Fat_Content_LF,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Fat_Content_reg,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,...,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,20.750000,0.007565,107.8622,1999.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,8.300000,0.038428,87.3198,2007.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,14.600000,0.099575,241.7538,1998.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,7.315000,0.015388,155.0340,2007.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,12.857645,0.118599,234.2300,1985.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5676,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,10.500000,0.013496,141.3154,1997.0
5677,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,7.600000,0.142991,169.1448,2009.0
5678,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,10.000000,0.073529,118.7440,2002.0
5679,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,15.300000,0.000000,214.6218,2007.0


In [78]:
y_train

0       3735.1380
1        443.4228
2       2097.2700
3        732.3800
4        994.7052
          ...    
8518    2778.3834
8519     549.2850
8520    1193.1136
8521    1845.5976
8522     765.6700
Name: Item_Outlet_Sales, Length: 8523, dtype: float64

In [94]:
############################ Regression with catboost ###################################

from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV, KFold

cbgm = CatBoostRegressor(random_state=24, logging_level='Silent')
params = {"max_depth":[None, 2, 3, 4],
              "n_estimators": [25, 50, 100],
              "learning_rate": np.linspace(0.01, 0.9, 10)}

kfold = KFold(n_splits=5, shuffle=True, random_state=24)
#param = {"alpha": np.linspace(0.001,5,10),"l1_ratio":np.linspace(0.1,0.9,9)}

gcv_cat = GridSearchCV(cbgm,cv = kfold, param_grid=params,scoring="neg_root_mean_squared_error", verbose=10)
gcv_cat.fit(X_train,y_train)
print(gcv_cat.best_score_)
print(gcv_cat.best_params_)
#cbgm.fit(X_train,y_train)
y_pred = gcv_cat.predict(X_test)


Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5; 1/120] START learning_rate=0.01, max_depth=None, n_estimators=25.......
[CV 1/5; 1/120] END learning_rate=0.01, max_depth=None, n_estimators=25;, score=-1457.074 total time=   0.2s
[CV 2/5; 1/120] START learning_rate=0.01, max_depth=None, n_estimators=25.......
[CV 2/5; 1/120] END learning_rate=0.01, max_depth=None, n_estimators=25;, score=-1515.521 total time=   0.2s
[CV 3/5; 1/120] START learning_rate=0.01, max_depth=None, n_estimators=25.......
[CV 3/5; 1/120] END learning_rate=0.01, max_depth=None, n_estimators=25;, score=-1475.028 total time=   0.1s
[CV 4/5; 1/120] START learning_rate=0.01, max_depth=None, n_estimators=25.......
[CV 4/5; 1/120] END learning_rate=0.01, max_depth=None, n_estimators=25;, score=-1507.782 total time=   0.2s
[CV 5/5; 1/120] START learning_rate=0.01, max_depth=None, n_estimators=25.......
[CV 5/5; 1/120] END learning_rate=0.01, max_depth=None, n_estimators=25;, score=-1571.859 total 

In [ ]:
help(GridSearchCV)

In [89]:
help(CatBoostRegressor)

Help on class CatBoostRegressor in module catboost.core:

class CatBoostRegressor(CatBoost)
 |  CatBoostRegressor(iterations=None, learning_rate=None, depth=None, l2_leaf_reg=None, model_size_reg=None, rsm=None, loss_function='RMSE', border_count=None, feature_border_type=None, per_float_feature_quantization=None, input_borders=None, output_borders=None, fold_permutation_block=None, od_pval=None, od_wait=None, od_type=None, nan_mode=None, counter_calc_method=None, leaf_estimation_iterations=None, leaf_estimation_method=None, thread_count=None, random_seed=None, use_best_model=None, best_model_min_trees=None, verbose=None, silent=None, logging_level=None, metric_period=None, ctr_leaf_count_limit=None, store_all_simple_ctr=None, max_ctr_complexity=None, has_time=None, allow_const_label=None, target_border=None, one_hot_max_size=None, random_strength=None, random_score_type=None, name=None, ignored_features=None, train_dir=None, custom_metric=None, eval_metric=None, bagging_temperature=No

In [95]:
y_pred[y_pred < 0] =0
df = pd.DataFrame({'Item_Identifier':dtest['Item_Identifier'], 'Outlet_Identifier': dtest['Outlet_Identifier'], 'Item_Outlet_Sales':y_pred})
df.to_csv(r'D:/CDAC MARCH 2024 AI/PML/assignments/4/submission_2.csv', index=False)

In [96]:
df = pd.DataFrame({'Item_Identifier':dtest['Item_Identifier'], 'Outlet_Identifier': dtest['Outlet_Identifier'], 'Item_Outlet_Sales':y_pred})